<a href="https://colab.research.google.com/github/Avradyooti/Practice/blob/main/Fynd_Round_2_Sec_B_Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd

# Load all datasets
sales = pd.read_csv("daily_sales.csv", parse_dates=["snapshot_date"])
inventory = pd.read_csv("daily_inventory.csv", parse_dates=["snapshot_date"])
products = pd.read_csv("products.csv")

# Merge sales + inventory by style_code and snapshot_date
data = pd.merge(sales, inventory, on=["style_code", "snapshot_date"], how="inner")

# Sort and calculate rolling 7-day sales
data = data.sort_values(by=["style_code", "snapshot_date"])
data["rolling_7d_sales"] = data.groupby("style_code")["units_sold"] \
    .transform(lambda x: x.rolling(window=7, min_periods=1).sum())

# Calculate days of cover
data["days_of_cover"] = data["on_hand_qty"] / (data["rolling_7d_sales"] / 7)

# Lifetime sales and latest on_hand_qty
lifetime_sales = data.groupby("style_code")["units_sold"].sum().rename("lifetime_units_sold")
latest_inventory = data.sort_values("snapshot_date").groupby("style_code").last()["on_hand_qty"].rename("latest_on_hand")

sell_through = (lifetime_sales / (lifetime_sales + latest_inventory)).rename("sell_through")

product_info = products[["style_code", "category"]].drop_duplicates()
summary = pd.concat([lifetime_sales, latest_inventory, sell_through], axis=1).reset_index()
summary = pd.merge(summary, product_info, on="style_code", how="left")

latest_doc = data.sort_values("snapshot_date").groupby("style_code").last()["days_of_cover"].rename("days_of_cover")
summary = pd.merge(summary, latest_doc, on="style_code", how="left")
print(summary["days_of_cover"].min())
print(summary["sell_through"].min())

at_risk = summary[
    (summary["days_of_cover"] < 14) &
    (summary["sell_through"] < 0.35)
].copy()

# Find other styles in same category with high inventory
transfer_candidates = summary[summary["days_of_cover"] > 21].groupby("category")["style_code"].apply(list).rename("transfer_candidates")

at_risk = pd.merge(at_risk, transfer_candidates, on="category", how="left")

# Assign action
def decide_action(row):
    candidates = row.get("transfer_candidates", [])
    if isinstance(candidates, list) and any(s != row["style_code"] for s in candidates):
        return "transfer"
    return "reorder"

at_risk["action"] = at_risk.apply(decide_action, axis=1)
final = at_risk[["style_code", "days_of_cover", "sell_through", "action"]]
print(final)
# Optional: export to CSV
# final.to_csv("stockout_alerts.csv", index=False)

4.5
0.673469387755102
Empty DataFrame
Columns: [style_code, days_of_cover, sell_through, action]
Index: []
